# Oracle PyDough Database connector

> ## 🚀 Initial Setup
>
> ---
>
> ### 1️⃣ Oracle Database
>
> You can connect to your **own Oracle database** using your credentials — for example, if you have **Oracle Database Software** or another local server running.
>
> ---
>
> ### 2️⃣ Docker Image (TPC-H Database)
>
> You can also test with our **pre-built Oracle TPC-H database** available on **Docker Hub**.
>
> #### 📋 Requirements
> - Make sure you have **Docker Desktop** installed and running.
>
> #### 📦 Pull and Run the Container
> ```bash
> docker run -d --name [CONTAINER_NAME]\
>      --platform linuxamd64 \
>     -e ORACLE_PWD=[PASSWORD] \
>     -p 1521:1521 \
>     bodoai1/pydough-oracle-tpch:latest
> ```
> - Replace `[CONTAINER_NAME]` with your preferred container name.  
> - Replace `[PASSWORD]` with your preferred password.
>
> *(Make sure the `1521` port is available and not being used by another container.)* 
> 
> ---
>
> #### 🔑 Environment Variables
> To connect to this database, use:
> ```env
> ORACLE_USERNAME=toch
> ORACLE_PASSWORD=[PASSWORD]
> ```
> *(Make sure `[PASSWORD]` matches the one you used when running the container.)*
>
> ---
>
> 💡 **Tip:**  
> Store these credentials in a `.env` file in your project directory for easy access and security.
>
> Example `.env` file:
> ```env
> ORACLE_USERNAME=root
> ORACLE_PASSWORD=mysecretpassword
> ```
>
>
> #### Deleting the container and image
> Once the tests have finished you can stop the container and delete it with the image using the following docker commands:
>```bash
>   docker stop [CONTAINER_NAME]
>   docker rm [CONTAINER_NAME]
>   docker rmi bodoai1/pydough-oracle-tpch
>```

> ## 🔌 Installing Oracle Connector
>
> Make sure to have the **`python-oracledb`** package installed:
>
> ---
>
> - **If you're working inside the repo**:
> ```bash
> pip install -e ".[oracle]"
> ```
>
> - **Or install the connector directly**:
> ```bash
> python -m pip install oracledb --upgrade
> ```

> ## Importing Required Libraries
>
> ---

In [ ]:
import pydough
import datetime
import os

> ## 🔑 Loading Credentials and connecting to Oracle with PyDough
>
> ---
>
> ### 1️⃣ Load Credentials from a Local `.env` File
> - The `.env` file contains your MySQL login details like:
>   ```env
>   ORACLE_PASSWORD=mypassword
>   ```
> - These are read in Python using:
>   ```python
>   import os
>   password = os.getenv("ORACLE_PASSWORD")
>   ```
>
> ---
>
> ### 2️⃣ Oracle-PyDough `connect_database()` Parameters
> - **`user`** *(required)*: Username for Oracle connection.  
> - **`password`** *(required)*: Password used for MySQL connection.  
> - **`service_name`** *(required)*: Oracle database service name.  
> - **`host`** *(optional)*: IP to access Oracle server. Default: `"localhost"` or `"127.0.0.1"`.  
> - **`port`** *(optional)*: Port number to access Oracle server. Default: `1521`.  
> - **`tcp_connect_timeout`** *(optional)*: Timeout in seconds for Oracle connection. Default: `3`.  
> - **`attempts`** *(optional)*: Number of times the connection is attempted. Default: `1`.  
> - **`delay`** *(optional)*: Seconds to wait before another connection attempt. Default: `2`.  
>
> ---
>
> ### 3️⃣ Connect to Oracle Using PyDough
> - `pydough.active_session.load_metadata_graph(...)`  
>   Loads a metadata graph mapping your Oracle schema (used for query planning/optimizations).  
> - `connect_database(...)`  
>   Uses the loaded credentials to establish a live connection to your Oracle database.
>
> ---
>
> **⚠️ Notes:**  
> - Ensure the `.env` exists and contains **all required keys**.  
> - The metadata graph path must point to a **valid JSON file** representing your schema.

In [ ]:
oracle_password = os.getenv("ORACLE_PASSWORD")
oracle_tpch_db = "tpch"
oracle_host = "127.0.0.1"
connection_timeout = 2
attempts = 2      
delay = 5.0       

pydough.active_session.load_metadata_graph("../../tests/test_metadata/sample_graphs.json", "TPCH")
pydough.active_session.connect_database("oracle", 
    user="tpch",
    password=oracle_password,
    service_name="FREEPDB1",
    host=oracle_host,
    tcp_connect_timeout=connection_timeout,
    attempts=attempts,
    delay=delay
)

> ## ✨ Enabling PyDough's Jupyter Magic Commands
>
> ---
>
> This step loads the **`pydough.jupyter_extensions`** module, which adds custom magic commands (like `%%pydough`) to your notebook.
>
> ---
>
> ### 📌 What These Magic Commands Do
> - **Write PyDough directly** in notebook cells using:
>   ```python
>   %%pydough
>   ```
> - **Automatically render** query results inside the notebook.
>
> ---
>
> ### 💻 How It Works
> This is a **Jupyter-specific feature** — the `%load_ext` command dynamically loads these extensions into your **current notebook session**:
> ```python
> %load_ext pydough.jupyter_extensions
> ```

In [ ]:
%load_ext pydough.jupyter_extensions

> ## 📊 Running TPC-H Query 1 with PyDough in Oracle
>
> ---
>
> This cell runs **TPC-H Query 1** using **PyDough**.
>
> ---
>
> ### 📝 What the Query Does
> - **Computes summary statistics**: sums, averages, and counts for orders.  
> - **Groups by**: `return_flag` and `line_status`.  
> - **Filters by**: a shipping date cutoff.  
>
> ---
>
> ### 📤 Output
> - `pydough.to_df(output)` converts the result to a **Pandas DataFrame**.  
> - This makes it easy to inspect and analyze results directly in Python.  
>
> ---
>

In [ ]:
%%pydough
# TPCH Q1
output = (lines.WHERE((ship_date <= datetime.date(1998, 12, 1)))
        .PARTITION(name="groups", by=(return_flag, status))
        .CALCULATE(
            L_RETURNFLAG=return_flag,
            L_LINESTATUS=status,
            SUM_QTY=SUM(lines.quantity),
            SUM_BASE_PRICE=SUM(lines.extended_price),
            SUM_DISC_PRICE=SUM(lines.extended_price * (1 - lines.discount)),
            SUM_CHARGE=SUM(
                lines.extended_price * (1 - lines.discount) * (1 + lines.tax)
            ),
            AVG_QTY=AVG(lines.quantity),
            AVG_PRICE=AVG(lines.extended_price),
            AVG_DISC=AVG(lines.discount),
            COUNT_ORDER=COUNT(lines),
        )
        .ORDER_BY(L_RETURNFLAG.ASC(), L_LINESTATUS.ASC())
)

pydough.to_df(output)